In [57]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
#from tensorflow.keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Data.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


<ipython-input-57-aa8f382bbd2e>:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-57-aa8f382bbd2e>:22: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 44s - 150ms/step - accuracy: 0.6417 - loss: 0.8268
144/144 - 4s - 25ms/step - accuracy: 0.6741 - loss: 0.7507
0.750653862953186
0.67409348487854
['loss', 'compile_metrics']


In [16]:
X_te="A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"
X_te = pd.Series(X_te.split())

In [17]:
X_te = X_te.apply(lambda x: x.lower())
X_te = X_te.apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [42]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(X_te.values)
X_p = tokenizer.texts_to_sequences(X_te.values)
batch_size = 32
X_p = pad_sequences(X_p)
y_predict= model.predict(X_p)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [46]:
print(y_predict)

[[0.39214003 0.31925628 0.2886036 ]
 [0.39135495 0.35303283 0.25561222]
 [0.4551091  0.27145484 0.27343598]
 [0.38383952 0.35258996 0.26357046]
 [0.38451567 0.314067   0.30141735]
 [0.33759928 0.37681255 0.2855882 ]
 [0.38688815 0.28496805 0.32814378]
 [0.48946804 0.26046926 0.25006267]
 [0.33759928 0.37681255 0.2855882 ]
 [0.3094345  0.34383819 0.3467273 ]
 [0.3323715  0.33487827 0.3327503 ]
 [0.35854405 0.3143336  0.32712236]
 [0.45653868 0.27410045 0.26936087]
 [0.41337729 0.3239085  0.2627142 ]
 [0.3326219  0.33741564 0.32996243]
 [0.28101528 0.2636259  0.45535883]
 [0.3921401  0.31925634 0.28860366]
 [0.38470036 0.2671263  0.3481734 ]
 [0.360558   0.33415806 0.30528387]
 [0.47224405 0.29044786 0.23730804]]


In [5]:
print(data)

                                                    text sentiment
0      rt nancyleegrahn how did everyone feel about t...   Neutral
1      rt scottwalker didnt catch the full gopdebate ...  Positive
2      rt tjmshow no mention of tamir rice and the go...   Neutral
3      rt robgeorge that carly fiorina is trending  h...  Positive
4      rt danscavino gopdebate w realdonaldtrump deli...  Positive
...                                                  ...       ...
13866  rt cappy_yarbrough love to see men who will ne...  Negative
13867  rt georgehenryw who thought huckabee exceeded ...  Positive
13868  rt lrihendry tedcruz as president i will alway...  Positive
13869  rt jrehling gopdebate donald trump says that h...  Negative
13870  rt lrihendry tedcruz headed into the president...  Positive

[13871 rows x 2 columns]


In [63]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 10]
    }

In [64]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy')

In [65]:
grid.fit(X, y)

TypeError: Cannot clone object '<Sequential name=sequential_1, built=True>' (type <class 'keras.src.models.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.